In [ ]:
import os
import sys

import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

sys.path.append('../')
from src.data_handler import DataHandler
from src.helper_functions import baseline_als

sns.set_theme()

In [ ]:
data_folder = os.path.join("C:\\", "Data")
figure_folder = os.path.join("C:\\", "5f12dd6ff78b78000186706b", "images", "ch3")

In [ ]:
deer_lt_handler = DataHandler(
    data_folder=data_folder,
    figure_folder=figure_folder,
    measurement_folder=os.path.join("171120_9002_NC60_LT", "NV4E06"),
    copy_measurement_folder_structure=False
)

In [ ]:
deer_rt_handler = DataHandler(
    data_folder=data_folder,
    figure_folder=figure_folder,
    measurement_folder=os.path.join("171219_9002_NC60_RT", "NV4E06"),
    copy_measurement_folder_structure=False
)

In [ ]:
epr_handler = DataHandler(
    data_folder=data_folder, 
    figure_folder=figure_folder, 
    measurement_folder=os.path.join("180504_NC60_EPR_CHROMATOGRAM_OSNABRUCK", "EPR"),
    copy_measurement_folder_structure=False
)

In [ ]:
sim_handler = DataHandler(
    data_folder=data_folder, 
    figure_folder=figure_folder, 
    measurement_folder="190719_NC60_SIMULATION",
    copy_measurement_folder_structure=False
)

In [ ]:
sim2_handler = DataHandler(
    data_folder=data_folder,
    figure_folder=figure_folder, 
    measurement_folder="20200701_NC60_SIMULATION",
    copy_measurement_folder_structure=False
)

# DEER

## Clean surface

In [ ]:
fig, ax0 = plt.subplots(figsize=(7, 1))

fname = "DEER_baseline"
d = deer_lt_handler.read_pkl(os.path.join(deer_lt_handler.data_folder_path, fname))

ax0.plot(d["x"]-273, d["y"] -d["base"] + 1, ".", mfc="white", color="C7", label="Experimental data")
ax0.plot(d["x"]-273, d["base"] - d["base"] + 1, linestyle="dotted", color="C7")
ax0.set_ylabel("$P(m_{s}=0)$")

ax0.legend(loc="lower right")
fig.tight_layout()

## DEER-DQ

In [ ]:
fig, ax = plt.subplots(figsize=(7, 5))

# Low temperature DEER
fname = "9002_1E-5_4K_NV4E06_deer_spectroscopy_DQT_2"
d = deer_lt_handler.read_pkl(os.path.join(deer_lt_handler.data_folder_path, fname))

center_freq = 271.5
lt_color = "C1"
base = baseline_als(d["y"], lam=1e6, p=0.95)

x = d["x"]-center_freq
y = d["y"]-base+1
# np.savetxt('DEER.txt', np.c_[x,y])

ax.errorbar(x, y, yerr=d["n"], fmt="o", mfc="white", capsize=2, color=lt_color, label="Experimental data")
ax.set_ylabel("$P(m_{s}=0)$")

detuning = (d["peaks"][0]-d["peaks"][1]) + 15.36

#print(d)
# Simulation
fname = "NC60_C60_T4K_LW6MHz.txt"
x2 = sim_handler.read_into_ndarray(os.path.join(sim_handler.data_folder_path, fname), usecols=0)
y2 = sim_handler.read_into_ndarray(os.path.join(sim_handler.data_folder_path, fname), usecols=1)

y2 = (-y2/max(y2))/140 + 1
x2 = x2*1000 - center_freq

center_freq = 270.7
# Simulation!
fname = "Plotted_C60.txt"
a = sim2_handler.read_into_ndarray(os.path.join(sim2_handler.data_folder_path, fname))
x2, y2 = a[:, 0], a[:, 1]
y2 = (y2/max(y2))/140 + 0.9928
ax.plot((x2[0:3400])*1e3-center_freq, y2[0:3400], "--", label="Spin-bath simulation", color="C4", zorder=3)

# Simulation!
fname = "Plotted_NC60.txt"
a = sim2_handler.read_into_ndarray(os.path.join(sim2_handler.data_folder_path, fname))
x2, y2 = a[:, 0], a[:, 1]
y2 = (y2/max(y2))/140 + 0.9928
ax.plot((x2[0:3400])*1e3-center_freq, y2[0:3400], "--", label="Endofullerene simulation",  color="C5", zorder=3)

center_freq = 271
# Simulation!
fname = "Plotted_LW4MHz2.txt"
a = sim2_handler.read_into_ndarray(os.path.join(sim2_handler.data_folder_path, fname))
x2, y2 = a[:, 0], a[:, 1]
diff = np.max(y2[0:3400]) - 1
ax.plot((x2[0:3400])*1e3-center_freq, y2[0:3400] - diff, "-", label="Combined simulation", color="C0", zorder=6)

ax.legend(loc="lower right")

fig.tight_layout()

## Room temperature

In [ ]:
fig, ax1 = plt.subplots(figsize=(7, 2))

# Room temperature DEER
fname = "9002_NC60_e-5_NV4E06_RT_deer_spectroscopy_1"
d = deer_rt_handler.read_pkl(os.path.join(deer_rt_handler.data_folder_path, fname))

center = d["peaks"][0]
rt_color = "C3"
rt_alpha = 1
a, b = 35, -19
shift = +0.001

ax1.errorbar(d["x"][a:b]-center, d["y"][a:b]+shift, yerr=d["n"][a:b], fmt="o", mfc="white", capsize=3, color=rt_color, label="Experimental data")
# a, b = 13
a, b = 140, -75
ax1.plot(d["x_int"][a:b]-center, d["fit_int"][a:b]+shift, color=rt_color, alpha=rt_alpha, zorder=2, linestyle="dotted")
ax1.set_ylabel("$P(m_{s}=0)$")

ax1.legend(loc="lower right")


In [ ]:
fig, (ax0, ax, ax1, ax2) = plt.subplots(nrows=4, sharex="col", figsize=(7, 7), gridspec_kw={'height_ratios':[2, 9, 4, 2]})

fname = "DEER_baseline"
d = deer_lt_handler.read_pkl(os.path.join(deer_lt_handler.data_folder_path, fname))

ax0.plot(d["x"]-273, d["y"] -d["base"] + 1, ".", mfc="white", color="C7", label="Experimental data")
ax0.plot(d["x"]-273, d["base"] - d["base"] + 1, linestyle="dotted", color="C7")
ax0.set_ylabel("$P(m_{s}=0)$")

ax0.legend(loc="lower right")

# Low temperature DEER
fname = "9002_1E-5_4K_NV4E06_deer_spectroscopy_DQT_2"
d = deer_lt_handler.read_pkl(os.path.join(deer_lt_handler.data_folder_path, fname))

center_freq = 271.5
lt_color = "C1"
base = baseline_als(d["y"], lam=1e6, p=0.95)

x = d["x"]-center_freq
y = d["y"]-base+1
# np.savetxt('DEER.txt', np.c_[x,y])

ax.errorbar(x, y, yerr=d["n"], fmt="o", mfc="white", capsize=2, color=lt_color, label="Experimental data")
ax.set_ylabel("$P(m_{s}=0)$")

detuning = (d["peaks"][0]-d["peaks"][1]) + 15.36

#print(d)
# Simulation
fname = "NC60_C60_T4K_LW6MHz.txt"
x2 = sim_handler.read_into_ndarray(os.path.join(sim_handler.data_folder_path, fname), usecols=0)
y2 = sim_handler.read_into_ndarray(os.path.join(sim_handler.data_folder_path, fname), usecols=1)

y2 = (-y2/max(y2))/140 + 1
x2 = x2*1000 - center_freq

# Room temperature DEER
fname = "9002_NC60_e-5_NV4E06_RT_deer_spectroscopy_1"
d = deer_rt_handler.read_pkl(os.path.join(deer_rt_handler.data_folder_path, fname))

center = d["peaks"][0]
rt_color = "C3"
rt_alpha = 1
a, b = 35, -19
shift = +0.001

ax1.errorbar(d["x"][a:b]-center, d["y"][a:b]+shift, yerr=d["n"][a:b], fmt="o", mfc="white", capsize=3, color=rt_color, label="Experimental data")
# a, b = 13
a, b = 140, -75
ax1.plot(d["x_int"][a:b]-center, d["fit_int"][a:b]+shift, color=rt_color, alpha=rt_alpha, zorder=2, linestyle="dotted")

# EPR measurement
fname = "S5-1_M10_overview"
de = epr_handler.read_pkl(os.path.join(epr_handler.data_folder_path, fname))
center = 336.25
norm = max(de["y"])
a, b = 1500, 2700 
x = (de["x"]-center)*28
y = de["y"]/norm
ax2.set(xlabel="$\omega_{RF} - 2μ_BB_0$ (MHz)", ylabel='$d\chi$"$/dB$')
ax2.set_ylabel(r"$\dfrac{d\chi^{\prime\prime}}{dB}$")

ax1.set_ylabel("$P(m_{s}=0)$")

colors = ["C3", "C2", "C0"]
indexes = [[0, a], [a, b], [b, -1]]

for p, color, idx in zip(de["peaks"], colors, indexes):
    idx1, idx2 = idx[0], idx[1]
    ax2.plot(x[idx1:idx2], y[idx1:idx2], color=color, linewidth=0.8)
    ax.axvline((de["x"][p]-center)*28, linestyle="--", color=color, alpha=0.5)
    ax1.axvline((de["x"][p]-center)*28, linestyle="--", color=color, alpha=0.5)

center_freq = 270.7
# Simulation!
fname = "Plotted_C60.txt"
a = sim2_handler.read_into_ndarray(os.path.join(sim2_handler.data_folder_path, fname))
x2, y2 = a[:, 0], a[:, 1]
y2 = (y2/max(y2))/140 + 0.9928
ax.plot((x2[0:3400])*1e3-center_freq, y2[0:3400], "--", label="Spin-bath simulation", color="C4", zorder=3)

# Simulation!
fname = "Plotted_NC60.txt"
a = sim2_handler.read_into_ndarray(os.path.join(sim2_handler.data_folder_path, fname))
x2, y2 = a[:, 0], a[:, 1]
y2 = (y2/max(y2))/140 + 0.9928
ax.plot((x2[0:3400])*1e3-center_freq, y2[0:3400], "--", label="Endofullerene simulation",  color="C5", zorder=3)

center_freq = 271
# Simulation!
fname = "Plotted_LW4MHz2.txt"
a = sim2_handler.read_into_ndarray(os.path.join(sim2_handler.data_folder_path, fname))
x2, y2 = a[:, 0], a[:, 1]
diff = np.max(y2[0:3400]) - 1
ax.plot((x2[0:3400])*1e3-center_freq, y2[0:3400] - diff, "-", label="Combined simulation", color="C0", zorder=6)

ax.legend(loc="lower right")

ax1.legend(loc="lower right")

ax0.set_title(r"a. Single spin EPR (LT-UHV) + Clean surface")
ax.set_title(r"b. Single spin EPR (LT-UHV) + ${}^{\mathrm{14}}$N@C${}_{\mathrm{60}}$")
ax1.set_title("c. Single spin EPR (NTP)")
ax2.set_title("d. Ensemble EPR")

fig.tight_layout()

# deer_lt_handler.save_figures(fig, "deer_spectroscopy", only_pdf=True)

# DEER Rabi

In [ ]:
#fig, ((ax, ax2), (ax1, ax3), (ax4, ax5)) = plt.subplots(nrows=3, ncols=2, figsize=(8, 7), gridspec_kw = {'width_ratios':[3, 1]})
fig, ((ax), (ax1)) = plt.subplots(nrows=2, ncols=1, figsize=(6, 6))

# Central hfi

# Central hfi low power
fname = "9002_1E-5_4K_NV4E06_deer_rabi_272MHz_-30dBm_long"
d = deer_lt_handler.read_pkl(fname)

shift = 1.7
ax.errorbar(d["x"], d["y_contrast"]+shift, yerr=d["n_contrast"], fmt="s", mfc="white", color="C1", alpha=1, capsize=3)
ax.plot(d["x_int"], d["fit_contrast_int"]+shift, color="C1", zorder=3, linewidth=2, label=r"$\nu_R$ (-30 dBm) = 1.39 MHz")
ax.set(xlabel="Evolution time ($\mathrm{\mu s}$)", ylabel=r"$P(m_s=0)$")

# Central hfi high power
fname = "9002_1E-5_4K_NV4E06_deer_rabi_272MHz_-20dBm_long"
d = deer_lt_handler.read_pkl(os.path.join(deer_lt_handler.data_folder_path, fname))

shift = -0.2
ax.errorbar(d["x"], d["y_contrast"]+shift, yerr=d["n_contrast"], fmt="o", mfc="white", capsize=3, color="C0")
ax.plot(d["x_int"], d["fit_contrast_int"]+shift, color="C0", zorder=3, linewidth=2, label=r"$\nu_R$ (-20 dBm) = 4.42 MHz")
ax.set(xlabel="τ (μs)", ylabel=r"$P(m_s=0)$")

# Upper hfi high power
fname = "9002_1E-5_4K_NV4E06_deer_rabi_282MHz_-11dBm"
d = deer_lt_handler.read_pkl(os.path.join(deer_lt_handler.data_folder_path, fname))

shift = -0.3
ax1.errorbar(d["x"], d["y_contrast"]+shift, yerr=d["n_contrast"], fmt="d", capsize=3, mfc="white", color="C2")
ax1.plot(d["x_int"], d["fit_contrast_int"]+shift, color="C2", zorder=3, linewidth=2, label=r"$\nu_R$ (-11 dBm) = 12.47 MHz")
ax1.set(xlabel=r"$\tau$ (μs)", ylabel=r"$P(m_s=0)$")
# ax1.set_yticks([0.7, 1.0, 1.3, 1.6])


ax.set_title("Central dip ($m_{I} = 0$)")
ax1.set_title(r"Upper dip ($m_{I} =+1$)")


ax.legend()
ax1.legend()

fig.tight_layout()

# deer_lt_handler.save_figures(fig, "deer_rabi", only_pdf=True)

In [ ]:
fname = "DEER_rabi_P_f"
d = deer_lt_handler.read_pkl(os.path.join(deer_lt_handler.data_folder_path, fname))

fig, ax = plt.subplots(figsize=(5, 2))

for pmw, ob, marker, color, ye, label in zip(d["P_mW"], d["obs"], d["markers"], d["colors"], d["yerr"], d["labels"]):
    ax.plot(np.sqrt(pmw), ob, marker, color=color, mfc="white", markersize=11, zorder=3)
    
ax.plot(np.sqrt(d["P_mW"]), d["fit"], "--", linewidth=2, color="C7", label="Linear fit", alpha=0.5, zorder=2)
ax.set(xlabel=r"$\sqrt{P} (\sqrt{mW})$", ylabel=r"$\nu_{Rabi}$ (MHz)")

ax.set_ylim([0, 15])
ax.legend(framealpha=0.0)
fig.tight_layout()

deer_lt_handler.save_figures(fig, "deer_rabi_freq_power", only_pdf=True)

# DEER Hahn

In [ ]:
fig, (ax) = plt.subplots(nrows=1, figsize=(5, 4))

# DEER Hahn LT
fname = "9002_1E-5_4K_NV4E06_deer_hahn_272MHz_-20dBm"
d = deer_lt_handler.read_pkl(os.path.join(deer_lt_handler.data_folder_path, fname))
lt_color = "C0"
ax.errorbar(d["x"], d["y_contrast"], yerr=d["n_contrast"], fmt="o", mfc="white", capsize=3, color=lt_color)
ax.plot(d["x"], d["fit_contrast"], color=lt_color, linewidth=1.25)
ax.set_yticks([0, 0.2, 0.4, 0.6])

# NV Hahn RT & LT
fname = "9002_NC60_e-5_NV4E06_RT_hahn_1"
d = deer_rt_handler.read_pkl(os.path.join(deer_rt_handler.data_folder_path, fname))
t2_nv_rt = round(d["popt"][2], 0)
t2_nv_rt_err = round(d["perr"][2], 0)
label = "Without $\mathrm{N@C_{60}}$, $T_2^{NV} = $" + f"{t2_nv_rt}" + "$\pm$" + f"{t2_nv_rt_err}" + " $\mathrm{\mu s}$"
#ax1.errorbar(d["x"], d["y_contrast"]-1, yerr=d["n_contrast"], fmt="s", markersize=4, color="C1", label=label)
#ax1.plot(d["x_int"], d["fit_contrast_int"]-1, color="C1", zorder=3)

fname = "9002_NV4E06_1e-4_hahn_2"
d = deer_lt_handler.read_pkl(os.path.join(deer_lt_handler.data_folder_path, fname))
t2_nv_rt = round(d["popt"][2], 1)
t2_nv_rt_err = round(d["perr"][2], 1)
label = "With $\mathrm{N@C_{60}}$, $T_2^{NV} = $" + f"{t2_nv_rt}" + "$\pm$" + f"{t2_nv_rt_err}" + " $\mathrm{\mu s}$"

ax.axvspan(t2_nv_rt/2, 2, alpha=0.3, facecolor='C3')
ax.set_xlim(0, 2)

ax.set_xlabel(r"$\tau$ (μs)")
ax.set_ylabel(r"$P(m_s = 0)$")
ax.set_xticks([0, 1, 2])

ax.annotate(r"Limited to NV $T_2/2$", (0.63, 0.70), size=10, xycoords='axes fraction', rotation=90, color="C3", va="center", ha="center")

ax.annotate("$T_2 \geq 1$ μs", (0.55, 0.7), xycoords='axes fraction', color="C0", size=13, ha='right', va='bottom')

fig.tight_layout()

deer_lt_handler.save_figures(fig, "deer_hahn", only_pdf=True)

In [ ]:
fig, (ax, ax1) = plt.subplots(nrows=2, sharex="col", figsize=(6, 4))

d = deer_lt_handler.read_pkl(os.path.join(deer_lt_handler.data_folder_path, "9002_1E-5_4K_NV4E06_deer_signal_evo_272MHz_-20dBm_reference"))

ax.plot(d["x"], d["y"], "s", mfc="white",label="w/o RF spin-flip pulse", color="C1")
ax.plot(d["x"], d["fit"], linewidth=1.25, color="C1", zorder=3)

d = deer_lt_handler.read_pkl(os.path.join(deer_lt_handler.data_folder_path, "9002_1E-5_4K_NV4E06_deer_signal_evo_272MHz_-20dBm_signal"))
ax.plot(d["x"], d["y"], "o", mfc="white",label="with RF spin-flip pulse", color="C0")
ax.plot(d["x"], d["fit"], linewidth=1.25, color="C0", zorder=3)
ax.legend()

d = deer_lt_handler.read_pkl(os.path.join(deer_lt_handler.data_folder_path, "9002_1E-5_4K_NV4E06_deer_signal_evo_272MHz_-20dBm_difference"))

ax1.plot(d["x"], d["y"], "d", mfc="white", color="C2")
ax1.plot(d["x"], d["fit"], linewidth=1.25, color="C2", zorder=3)
a = 10
ax.set_ylabel('Norm. $P(m_s= 0)$')
ax1.set_ylabel("$\Delta P(m_s = 0)$")
ax1.set_xlabel('$τ$ (μs)')

print("slope/coupling strength (MHz)=", d["popt"][1], "error", d["perr"][1])
r = (52/d["popt"][1])**(1/3)
print("distance", r, "error", 1/3 * d["perr"][1]/d["popt"][1] * r)

ax1.axvspan(0.4, 1.3, alpha=0.3, facecolor ='C2', label=r"N@C$_{60}$ dominated")
ax1.axvspan(2.5, 8, alpha=0.3, facecolor ='C3', label="Spin-bath dominated")

ax1.legend()

fig.tight_layout()

deer_lt_handler.save_figures(fig, "deer_dominated", only_pdf=True)